In [ ]:
import pickle
import itertools
import numpy as np
from scipy import spatial
from scipy.stats import norm
import nltk.data
from nltk import pos_tag
from nltk.corpus import wordnet as wn
from nltk.corpus import reuters
from nltk. corpus import gutenberg
from nltk.corpus import brown
from nltk.tokenize import sent_tokenize
from gensim.models import KeyedVectors
from keras.layers import Input, Dense, Lambda, Layer
from keras.callbacks import ModelCheckpoint
from keras.models import Model
from keras import backend as K
from keras import metrics
from gensim import models
from sklearn.model_selection import train_test_split

### Load Google News Word2Vec model

In [ ]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz' # from above
word2vec = models.KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

--2020-08-07 05:42:19--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.187.21
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.187.21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/root/input/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  68.0MB/s    in 22s     

2020-08-07 05:42:42 (71.2 MB/s) - ‘/root/input/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


### Functions to process sentences

In [ ]:
def split_into_sent (text):
    strg = ''
    for word in text:
        strg += word
        strg += ' '
    strg_cleaned = strg.lower()
    for x in ['\xd5d','\n','"',"!", '#','$','%','&','(',')','*','+',',','-','/',':',';','<','=','>','?','@','[','^',']','_','`','{','|','}','~','\t']:
        strg_cleaned = strg_cleaned.replace(x, '')
    sentences = sent_tokenize(strg_cleaned)
    return sentences


# In[4]:


def vectorize_sentences(sentences):
    vectorized = []
    for sentence in sentences:
        byword = sentence.split()
        concat_vector = []
        for word in byword:
            try:
                concat_vector.append(w2v[word])
            except:
                pass
        vectorized.append(concat_vector)
    return vectorized

### Import Data File

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving reddit.csv to reddit.csv


# Data Preprocessing

In [ ]:
import pandas as pd
#Data Input and Preprocessing which is data specific
data = pd.read_csv('reddit.csv')
data_mod = pd.DataFrame([[]])

for row in range(len(data["id"])):
    #print(data["id"][row])
    data["id"][row]=(data["id"][row].splitlines())
    for num in range(len(data["id"][row])):
        strin=''.join(data["id"][row][num].split())
        newstr='';
        for i in range(len(strin)-2):
            newstr+=strin[i+2]
        data["id"][row][num]=newstr

for row in range(len(data["hate_speech_idx"])):
        if data["hate_speech_idx"][row]=='n/a':
            data["hate_speech_idx"][row]="[0]"

for row in range(len(data["hate_speech_idx"])):
        if data["hate_speech_idx"][row]=='n/a':
            data["hate_speech_idx"][row]="[0]"

for row in range(len(data["hate_speech_idx"])):
        if isinstance(data["hate_speech_idx"][row],float):
            data["hate_speech_idx"][row]="[0]"  

for row in range(len(data["response"])):
        if isinstance(data["response"][row],float):
            data["response"][row]='This is normal'
            
for row in range(len(data["text"])):
    #print(data["id"][row])
    data["text"][row]=(data["text"][row].splitlines())
    for num in range(len(data["text"][row])):
        strin=' '.join(data["text"][row][num].split())
        newstr='';
        for i in range(len(strin)-2):
            newstr+=strin[i+2]
        data["text"][row][num]=newstr
    
    data["hate_speech_idx"][row]=data["hate_speech_idx"][row].strip('][').split(', ');
    data["hate_speech_idx"][row]=list(map(int, data["hate_speech_idx"][row]))
    data["response"][row] = data["response"][row].replace("\"","\'")
    data["response"][row] = data["response"][row].replace("[\'","")
    data["response"][row] = data["response"][row].replace("\']","")
    data["response"][row] = data["response"][row].split('\', \'');
    
    for idx in data["hate_speech_idx"][row]:
        if idx!=0 and idx<=len(data["text"][row]):
            bad=idx-1;
#            print(bad)
            for resp in data["response"][row]:
                newrow=pd.Series([data["text"][row][bad],resp])
                rowdf=pd.DataFrame([newrow])
                data_mod=pd.concat([data_mod,rowdf],ignore_index=True)
                #print("done")


In [ ]:
df=data_mod
df = df.iloc[1:]
df.head()

,0,1
1,A subsection of retarded Hungarians? Ohh boy....,I don't see a reason why it's okay to insult o...
2,A subsection of retarded Hungarians? Ohh boy....,Language people. You can express your ire resp...
3,A subsection of retarded Hungarians? Ohh boy....,Stop using mental impairments for insults on p...
4,ah a liberal has slipped in. you can tell bec...,Persons with disabilities is the accepted term...
5,ah a liberal has slipped in. you can tell bec...,Avert from using mental impairments as a way t...


##SPLIT DATA INTO TRAIN AND TEST

In [ ]:
data_train, data_test = train_test_split(df, test_size=0.20, random_state=42)
train_x = pd.DataFrame(data_train[0])
train_y = pd.DataFrame(data_train[1])
test_x  = pd.DataFrame(data_test[0])
test_y  = pd.DataFrame(data_test[1])
type(test_y)

pandas.core.frame.DataFrame

In [ ]:
import nltk
import re
import os
import collections
import string
nltk.download('punkt')
nltk.download('stopwords')

def remove_punct(text):
    text_nopunct = ''
    text_nopunct = re.sub('['+string.punctuation+']', '', text)
    return text_nopunct

train_x[0] = train_x[0].apply(lambda x: remove_punct(x))
train_y[1] = train_y[1].apply(lambda x: remove_punct(x))
test_x[0] = test_x[0].apply(lambda x: remove_punct(x))
test_y[1] = test_y[1].apply(lambda x: remove_punct(x))
print(test_y.head())


from nltk import word_tokenize, WordNetLemmatizer
trainx_tokens = [word_tokenize(sen) for sen in train_x[0]]
trainy_tokens = [word_tokenize(sen) for sen in train_y[1]]
testx_tokens = [word_tokenize(sen) for sen in test_x[0]]
testy_tokens = [word_tokenize(sen) for sen in test_y[1]]

def lower_token(tokens): 
    return [w.lower() for w in tokens]    
    
lower_tokensx = [lower_token(token) for token in trainx_tokens]
lower_tokensy = [lower_token(token) for token in trainy_tokens]

print((lower_tokensy[0]))

from nltk.corpus import stopwords
stoplist = stopwords.words('english')

def remove_stop_wordsy(tokens):
    return [word for word in trainy_tokens if word not in stoplist]

def remove_stop_wordsx(tokens):
    return [word for word in trainx_tokens if word not in stoplist]

filtered_wordsx = [remove_stop_wordsx(sen) for sen in lower_tokensx] 
filtered_wordsy= [remove_stop_wordsy(sen) for sen in lower_tokensy] 

data_train_x=[[]]
data_train_y=[[]]

for i in range(len(filtered_wordsx[0])):
    x=filtered_wordsx[0][i]
    data_train_x.append(x)
for i in range(len(filtered_wordsy[0])):
    x=filtered_wordsy[0][i]
    data_train_y.append(x)

data_train_y.remove([])
data_train_x.remove([])
print((data_train_y[0]))

resultx = [' '.join(sen) for sen in data_train_x]
resulty = [' '.join(sen) for sen in data_train_y]
#print(len(resultx))
print(resulty[0])
train_x = pd.DataFrame(resultx,columns=['Text'])
train_y = pd.DataFrame(resulty,columns=['Response'])
print(train_y.head())
print(train_x.head())


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
                                                       1
4451   Your post uses speech that denigrates sex and ...
7855   I really hope you know the history that goes i...
7051   Immigrants and muslims are humans and are not ...
12869  Retard is a pejorative and hateful word that d...
7243   Well men and women can be moody but neither ar...
['consider', 'using', 'different', 'language', 'in', 'the', 'future', 'that', 'does', 'not', 'undermine', 'your', 'points', 'with', 'hatred']
['Consider', 'using', 'different', 'language', 'in', 'the', 'future', 'that', 'does', 'not', 'undermine', 'your', 'points', 'with', 'hatred']
Consider using different language in the future that does not undermine your points with hatred
                                            Resp

In [ ]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_comments, generate_missing=False):
    embeddings = clean_comments['tokens'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

train_x = []

#print(text[0:2])
import tensorflow as tf
vect = vectorize_sentences(resultx)
p=0
data=[]
y=np.zeros((300),dtype=float)
print(len(vect[0]))
for x in vect:
    if len(x)<50:
        t=50-len(x)
        for i in range(t):
            x.append(y)
    elif len(x)>50:
        x=x[0:50]
    if(len(x)==50):
        data.append(x)
    p=p+1
    if p==500:
        break
for x in data:
    y=(list(itertools.chain.from_iterable(x)))
    train_x.append(y)
print(len(train_x[0]))

train_y = []
vect = vectorize_sentences(resulty)
p=0
data=[]
y=np.zeros((300),dtype=float)
print(len(vect[0]))
for x in vect:
    if len(x)<50:
        t=50-len(x)
        for i in range(t):
            x.append(y)
    elif len(x)>50:
        x=x[0:50]
    if(len(x)==50):
        data.append(x)
    p=p+1
    if p==500:
        break
for x in data:
    y=(list(itertools.chain.from_iterable(x)))
    train_y.append(y)
print(len(train_y[0]))

0
15000
0
15000


In [ ]:
print((train_x[0][0]))
print((train_y[0][0]))
train_x = np.array(train_x)
train_y = np.array(train_y)
print(len(train_x))
print(len(train_y))
text_x=train_x[0:10]
test_y=train_y[0:10]
print(test_x)

0.0
0.0
500
500
                                                       0
4451    RToronto banned me for supporting free speech...
7855                             Ur a niggerfaggot harry
7051    Goddamn this is retarded Listen I know that y...
12869                           LMAO a retard and a cuck
7243                                Most women are cunts
...                                                  ...
11390   Thats just retard blaming not victim blaming ...
950     Dont insult retards like that even they have ...
5563                    Save this shit for ifunny faggot
7915    Im convinced these have to be leftist shit po...
8595                                  What a gradeA cunt

[2858 rows x 1 columns]


### VAE

In [ ]:
batch_size = 500
original_dim = 3000
latent_dim = 1000
intermediate_dim = 1200
epochs = 200
epsilon_std = 1.0

x = Input(batch_shape=(batch_size, original_dim))
h = Dense(intermediate_dim, activation='relu')(x)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0.,
                              stddev=epsilon_std)
    return z_mean + K.exp(z_log_var / 2) * epsilon

z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])
decoder_h = Dense(intermediate_dim, activation='relu')
decoder_mean = Dense(original_dim, activation='sigmoid')
h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)

# placeholder loss
def zero_loss(y_true, y_pred):
    return K.zeros_like(y_pred)

# Custom loss layer
class CustomVariationalLayer(Layer):
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomVariationalLayer, self).__init__(**kwargs)

    def vae_loss(self, x, x_decoded_mean):
        xent_loss = original_dim * metrics.binary_crossentropy(x, x_decoded_mean)
        kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        return K.mean(xent_loss + kl_loss)

    def call(self, inputs):
        x = inputs[0]
        x_decoded_mean = inputs[1]
        loss = self.vae_loss(x, x_decoded_mean)
        self.add_loss(loss, inputs=inputs)
        # we don't use this output, but it has to have the correct shape:
        return K.ones_like(x)

loss_layer = CustomVariationalLayer()([x, x_decoded_mean])
vae = Model(x, [loss_layer])
vae.compile(optimizer='rmsprop', loss=[zero_loss])


#train
vae.fit(train, train,
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(test, test), callbacks=cp)

# build a model to project inputs on the latent space
encoder = Model(x, z_mean)

# build a generator that can sample from the learned distribution
decoder_input = Input(shape=(latent_dim,))
_h_decoded = decoder_h(decoder_input)
_x_decoded_mean = decoder_mean(_h_decoded)
generator = Model(decoder_input, _x_decoded_mean)

### Generating texts from Latent Space

In [ ]:
def sent_parse(sentence, mat_shape):
    data_concat = []
    word_vecs = vectorize_sentences(sentence)
    for x in word_vecs:
        data_concat.append(list(itertools.chain.from_iterable(x)))
    zero_matr = np.zeros(mat_shape)
    zero_matr[0] = np.array(data_concat)
    return zero_matr

# input: original dimension sentence vector
# output: text
def print_sentence_with_w2v(sent_vect):
    word_sent = ''
    tocut = sent_vect
    for i in range (int(len(sent_vect)/300)):
        word_sent += w2v.most_similar(positive=[tocut[:300]], topn=1)[0][0]
        word_sent += ' '
        tocut = tocut[300:]
    print(word_sent)


# input: encoded sentence vector
# output: encoded sentence vector in dataset with highest cosine similarity
def find_similar_encoding(sent_vect):
    all_cosine = []
    for sent in sent_encoded:
        result = 1 - spatial.distance.cosine(sent_vect, sent)
        all_cosine.append(result)
    data_array = np.array(all_cosine)
    maximum = data_array.argsort()[-3:][::-1][1]
    new_vec = sent_encoded[maximum]
    return new_vec

# input: two written sentences, VAE batch-size, dimension of VAE input
# output: the function embeds the sentences in latent-space, and then prints their generated text representations
# along with the text representations of several points in between them
def sent_2_sent(sent1,sent2, batch, dim):
    a = sent_parse([sent1], (batch,dim))
    b = sent_parse([sent2], (batch,dim))
    encode_a = encoder.predict(a, batch_size = batch)
    encode_b = encoder.predict(b, batch_size = batch)
    test_hom = hom_shortest(encode_a[0], encode_b[0], 5)
    
    for point in test_hom:
        p = generator.predict(np.array([point]))[0]
        print_sentence(p)


###Sample_Texts_generated

In [ ]:
print_sentence_with_w2v(train_x[1])
print_sentence_with_w2v(train_x[2])